# Median of an Integer Stream
Design a data structure that supports adding integers from a data stream and retrieving the median of all elements received at any point.
- `add(num: int) -> None`: adds an integer to the data structure.
- `get_median() -> float`: returns the median of all integers so far.

**Example:**
Input: [add(3), add(6), get_median(), add(1), get_median()]<br/>
Output: [4.5, 3.0]

Explanation:<br/>
| Operation       | Sorted Data Structure | Median  |
|-----------------|-----------------------|---------|
| add(3)         | [3]                    | -       |
| add(6)         | [3, 6]                 | -       |
| get_median()   | [3, 6]                 | 4.5     |
| add(1)         | [1, 3, 6]              | -       |
| get_median()   | [1, 3, 6]              | 3.0     |


**Constraints:**
- At least one value will have been added before `get_median()` is called.


## Intuition

## **Understanding the Median**
The **median** is the middle value in a sorted list of numbers.  
- If the list has an **odd** length, the median is the middle element.  
- If the list has an **even** length, the median is the average of the two middle elements.  

### **Challenge: Handling a Stream of Numbers**
Since numbers arrive **in an unsorted order**, maintaining a sorted list dynamically is inefficient.  
Instead, we **only need to ensure that the middle values are in their correct positions**.

---

## **Heap-Based Approach**
A **min-heap and a max-heap** allow efficient tracking of median values:

- **Max-heap (left_half)** stores the **smaller** half of numbers.  
  - The **largest value** in this half represents the **first median**.  
- **Min-heap (right_half)** stores the **larger** half of numbers.  
  - The **smallest value** in this half represents the **second median**.  

If there are an **odd** number of elements, the **max-heap** holds the extra element (i.e., the median).

---

## **Populating and Balancing the Heaps**
To maintain the correct heap structure:

1. **Insert new numbers**  
   - If the number is **≤ max-heap’s top**, add it to the max-heap.  
   - Otherwise, add it to the min-heap.

2. **Ensure balance** (Rule: max-heap can have at most **one** extra element)
   - If **max-heap's size** > min-heap's size + 1 → Move max-heap’s top to min-heap.  
   - If **min-heap's size** > max-heap's size → Move min-heap’s top to max-heap.  

This **guarantees** that:
- **All values in left_half ≤ all values in right_half**.
- **Heap sizes stay nearly equal**, with the max-heap holding an extra element in odd cases.

---

## **Retrieving the Median**
With the middle elements stored at the **tops of the heaps**, finding the median is straightforward:

1. **Odd total elements** → The median is the **top of max-heap**.
2. **Even total elements** → The median is the **average of both heaps' tops**.

In [1]:
import heapq

class MedianOfAnIntegerStream:
    def __init__(self):
        self.left_half = []
        self.right_half = []

    def add(self, num: int) -> None:
        if not self.left_half or num <= -self.left_half[0]:
            heapq.heappush(self.left_half, -num)

            if len(self.left_half) > len(self.right_half) + 1:
                heapq.heappush(self.right_half, -heapq.heappop(self.left_half))
        
        else:
            heapq.heappush(self.right_half, num)

            if len(self.left_half) < len(self.right_half):
                heapq.heappush(self.left_half, -heapq.heappop(self.right_half))

    def get_median(self) -> float:
        if len(self.left_half) == len(self.right_half):
            return (-self.left_half[0] + self.right_half[0]) / 2.0
        
        return -self.left_half[0]

### **Time Complexity**
- **`add(num)` → O(log n)**
  - Inserting a number into a heap takes **O(log n)** time.
  - Rebalancing involves at most **one pop and one push**, both **O(log n)** operations.
  - The total complexity remains **O(log n)**.

- **`get_median()` → O(1)**
  - Accessing the top of the heap takes **O(1)** time.

### **Space Complexity**
- **O(n)** → Since both heaps together store all `n` elements.